In [2]:
import os
import sys
import re
import json
import numpy as np
import pandas as pd
from collections import defaultdict

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../onmt'))
if module_path not in sys.path:
    sys.path.append(module_path)

import kp_evaluate
import onmt.keyphrase.utils as utils

import seaborn as sns
import matplotlib.pyplot as plt
import scipy

from nltk.stem.porter import PorterStemmer

In [4]:
dataset_names = ['kp20k_valid2k']
# dataset_names = ['inspec', 'krapivin', 'nus', 'semeval', 'kp20k', 'duc', 'magkp', 'stackexchange']

# json_base_dir = '/Users/memray/project/kp/OpenNMT-kpg/data/keyphrase/json/' # path to the json folder
json_base_dir = '/zfs1/hdaqing/rum20/kp/data/kp/json' # path on CRC

dataset_examples = {}
    
for dataset_name in dataset_names:
    dataset_examples[dataset_name] = []
    print(dataset_name)

    input_json_path = os.path.join(json_base_dir, dataset_name, 'test.json')
    
    with open(input_json_path, 'r') as input_json:
        for json_line in input_json:
            ex_dict = json.loads(json_line)

            if dataset_name == 'stackexchange':
                ex_dict['abstract'] = ex_dict['question']
                ex_dict['keywords'] = ex_dict['tags']            
                del ex_dict['question']
                del ex_dict['tags']

            keywords = ex_dict['keywords']
            ex_dict['fulltext'] = ''

            if isinstance(keywords, str):
                keywords = keywords.split(';')
                ex_dict['keywords'] = keywords
            keywords = [k.strip() for k in keywords]
            ex_dict['keywords'] = keywords
            
            dataset_examples[dataset_name].append(ex_dict)


kp20k_valid2k


In [5]:
pred_paths = {
    'bart-o2s': '/zfs1/hdaqing/rum20/kp/fairseq-kpg/exps/kp/bartFT_presabs_kp20k_100k_rerun/outputs/beamsearch-width_50-maxlen_40/pred/checkpoint_step_45000-data_kp20k_valid2k_test.pred',
    'tf-o2s': '/zfs1/hdaqing/rum20/kp/fairseq-kpg/exps/kp/transformer_presabs_kp20k/outputs/beamsearch-width_50-maxlen_40/pred/checkpoint_step_95000-data_kp20k_valid2k_test.pred',
    'bart-o2o': '/zfs1/hdaqing/rum20/kp/fairseq-kpg/exps/kp_o2o/bartFT_one2one_kp20k_100k/outputs/beamsearch-width_128-maxlen_8/pred/checkpoint_step_90000-data_kp20k_valid2k_test.pred',
    }

pred_results = {}

for exp_name, pred_path in pred_paths.items():
    pred_results[exp_name] = []
    print(exp_name)

    with open(pred_path, 'r') as output_json:
        for json_line in output_json:
            ex_dict = json.loads(json_line)
            pred_results[exp_name].append(ex_dict)


bart-o2s
tf-o2s
bart-o2o


In [14]:
dataset_name = 'kp20k_valid2k'
exp_name = 'bart-o2s'
num_pred = 15

data_count = 0
for ex_data, pred_data in zip(dataset_examples[dataset_name], pred_results[exp_name]):
#     print(ex_data)
#     print(pred_data)
    data_count += 1
    
    if data_count < 12:
        continue
        
    print('title:', ex_data['title'])
    print('abstract:', ex_data['abstract'])
    print('GT-keywords [%d]' % len(ex_data['keywords']))
    for gt_id, gt in enumerate(ex_data['keywords']):
        print('\t [%d] ' % gt_id, gt)
    
    print()
    print('PRED-keywords [%d/%d]' % (num_pred, len(pred_sents)))
    pred_sents = pred_data['pred_sents']
    for pred_id, pred in enumerate(pred_sents[: min(num_pred, len(pred_sents))]):
        print('\t [%d] ' % pred_id, pred)
    break

title: A structural approach to reversible computation
abstract: Reversibility is a key issue in the interface between computation and physics, and of growing importance as miniaturization progresses towards its physical limits. Most foundational work on reversible computing to date has focussed on simulations of low-level machine models. By contrast, we develop a more structural approach. We show how high-level functional programs can be mapped compositionally (i.e. in a syntax-directed fashion) into a simple kind of automata which are immediately seen to be reversible. The size of the automaton is linear in the size of the functional term. In mathematical terms, we are building a concrete model of functional computation. This construction stems directly from ideas arising in Geometry of Interaction and Linear Logic-but can be understood without any knowledge of these topics. In fact, it serves as an excellent introduction to them. At the same time, an interesting logical delineation 